# Import mstrio

To use mstrio, you need to import mstrio library. Yon can find instructions on https://github.com/MicroStrategy/mstrio-py

We don't have python native class for Security Filter yet, but you use the generic wrapper and python dictionary object. 

In [ ]:
from mstrio.connection import Connection
import getpass

In [ ]:
base_url = "https://env-248739.customer.cloud.microstrategy.com/MicroStrategyLibrary/api"
mstr_username = "mstr"
mstr_password = getpass.getpass()
project_id = "B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn = Connection(base_url, mstr_username, mstr_password, project_id=project_id)

# Security Filters

## Search for security filters
Type 58 is security filter

In [ ]:
res = conn.session.get(url=conn.base_url + "/api/searches/results?name=&pattern=4&type=58&getAncestors=false&offset=0&limit=-1")

In [ ]:
res.json()

Get the users and user groups that the specified security filter is applied to

In [ ]:
securityFilterID = "0255170B4A79EBF2BAB2D88D824622F3"
res = conn.session.get(url=conn.base_url + "/api/securityFilters/"+securityFilterID+"/members?offset=0&limit=-1")

In [ ]:
res.json()

## Create Security Filter

Ref to: https://www2.microstrategy.com/producthelp/Current/RESTSDK/Content/create_security_filter_object.htm

### 1. Create Change Set

To create security filter, you need to create a changeset. 

In [ ]:
ChangeSets_EndPoint = "/api/model/changesets"

In [ ]:
res = conn.session.post(url=conn.base_url + ChangeSets_EndPoint)

In [ ]:
changesetID = res.json()['id']

### 2. Create Security Filter 

Add changesetId too the header

In [ ]:
conn.session.headers["X-MSTR-MS-Changeset"] = changesetID

In [ ]:
conn.session.headers

Define the security filter defintion. Check the scheme in API doc. https://demo.microstrategy.com/MicroStrategyLibrary/api-docs/index.html#/Security%20Filters/ms-postSecurityFilter

In [ ]:
data = {
  "information": {
    "subType": "md_security_filter",
    "name": "Year > 2015", 
    "destinationFolderId": "98FE182C2A10427EACE0CD30B6768258"
  },
  "qualification": {
    "tree": {
      "type": "predicate_form_qualification",
      "predicateTree": {
        "function": "greater",
        "parameters": [
          {
            "parameterType": "constant",
            "constant": {
              "type": "double",
              "value": "2015.0"
            }
          }
        ],
        "attribute": {
          "objectId": "8D679D5111D3E4981000E787EC6DE8A4",
          "subType": "attribute",
          "name": "Year"
        },
        "form": {
          "objectId": "45C11FA478E745FEA08D781CEA190FE5",
          "subType": "attribute_form_system",
          "name": "ID"
        }
      }
    }
  },
  "topLevel": [],
  "bottomLevel": []
}

In [ ]:
data

Send post request to create a security filter. 

In [ ]:
import json
res = conn.session.post(url=conn.base_url + "/api/model/securityFilters",data=json.dumps(data))

In [ ]:
res.status_code

In [ ]:
securityFilter = res.json()

In [ ]:
securityFilterId = securityFilter["information"]["objectId"]

In [ ]:
securityFilterId

### 3. Commit changeset

In [ ]:
res = conn.session.post(url=conn.base_url + "/api/model/changesets/"+changesetID+"/commit")

In [ ]:
res.status_code

In [ ]:
res.json()

### 4. Check if security filter is created

In [ ]:
res = conn.session.get(url=conn.base_url + "/api/model/securityFilters/"+securityFilterId)

In [ ]:
res.json()

### 5. Delete a changeset

In [ ]:
res = conn.session.delete(url=conn.base_url + "/api/model/changesets/"+changesetID)

In [ ]:
res.status_code